In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import os
from qgsw.utils.sorting import sort_files
from qgsw.run_summary import RunSummary
from qgsw.models.qg.alpha import ChangingCoefficient


In [ ]:
ROOT = Path(os.path.abspath('')).parent

# Kernel : 30

In [ ]:
%matplotlib inline

coef = ChangingCoefficient.from_file(ROOT.joinpath("data/coefficients.npz"))
coef._kernel_width = 30
coef._coef_interpolation = coef._interpolate_coefs(coef._times, coef._coefs)
coef.show(False)

In [ ]:
%matplotlib widget

folder_alpha = ROOT.joinpath("output/g5k/changing_kernel_30")
run_alpha = RunSummary.from_file(folder_alpha.joinpath("_summary.toml"))
steps_alpha, files_alpha = sort_files(list(folder_alpha.glob(f"{run_alpha.configuration.model.prefix}*.npz")),run_alpha.configuration.model.prefix,".npz")
folder_1l = ROOT.joinpath("output/g5k/one_layer_baroclinic_100km")
run_1l = RunSummary.from_file(folder_1l.joinpath("_summary.toml"))
steps_1l, files_1l = sort_files(list(folder_1l.glob(f"{run_1l.configuration.model.prefix}*.npz")),run_1l.configuration.model.prefix,".npz")
folder_2l = ROOT.joinpath("output/g5k/two_layers_baroclinic_100km")
run_2l = RunSummary.from_file(folder_2l.joinpath("_summary.toml"))
steps_2l, files_2l = sort_files(list(folder_2l.glob(f"{run_2l.configuration.model.prefix}*.npz")),run_2l.configuration.model.prefix,".npz")

if not (run_1l.dt == run_alpha.dt == run_2l.dt):
    print("Timesteps are not matching.")
if not (run_1l.duration == run_alpha.duration == run_2l.duration):
    print("Duration are not matching.")

plt.ion()
fig = plt.figure(figsize = (18,18))

layer_1 = fig.add_subplot(3,3,1)
layer_2 = fig.add_subplot(3,3,2)
layer_3 = fig.add_subplot(3,3,3)
comparison_1 = fig.add_subplot(3,3,4)
comparison_2 = fig.add_subplot(3,3,5)
comparison_3 = fig.add_subplot(3,3,6)

rmses = fig.add_subplot(3,1,3)

rmse_1l_2l_30 = []
rmse_1l_alpha_30 = []
rmse_2l_alpha_30 = []

freq_save = len(files_alpha) // 10 + 1

for i in range(len(files_alpha)):
    if not (steps_alpha[i] == steps_1l[i] == steps_2l[i]):
        msg = f"Impossible to match steps {steps_alpha[i]},{steps_1l[i]} and {steps_2l[i]}."
        raise ValueError(msg)
    
    layer_alpha = np.load(files_alpha[i])["omega"][0,0]
    layer_1l = np.load(files_1l[i])["omega"][0,0]
    layer_2l = np.load(files_2l[i])["omega"][0,0]

    diff_1l_2l = np.power(layer_1l - layer_2l,2)
    diff_1l_alpha = np.power(layer_1l - layer_alpha,2)
    diff_2l_alpha = np.power(layer_2l - layer_alpha,2)

    rmse_1l_2l_30.append(np.sqrt(np.sum(diff_1l_2l)) )
    rmse_1l_alpha_30.append(np.sqrt(np.sum(diff_1l_alpha)))
    rmse_2l_alpha_30.append(np.sqrt(np.sum(diff_2l_alpha)) )

    
    fig.suptitle(f"t: {steps_alpha[i]*run_alpha.dt}s/{steps_alpha[-1]*run_alpha.dt}s")

    layer_1.cla()
    layer_2.cla()
    layer_3.cla()
    comparison_1.cla()
    comparison_2.cla()
    comparison_3.cla()

    rmses.cla()

    layer_1.set_title("One Layer")
    layer_1.imshow(layer_1l)
    layer_2.set_title("Modified Layer")
    layer_2.imshow(layer_alpha)
    layer_3.set_title("Two Layers")
    layer_3.imshow(layer_2l)
    comparison_1.set_title(fr"1l - 2l: {rmse_1l_2l_30[-1]:.2f}")
    comparison_1.imshow(np.sqrt(diff_1l_2l))
    comparison_2.set_title(rf"1l - $\alpha$: {rmse_1l_alpha_30[-1]:.2f}")
    comparison_2.imshow(np.sqrt(diff_1l_alpha))
    comparison_3.set_title(fr"2l - $\alpha$: {rmse_2l_alpha_30[-1]:.2f}")
    comparison_3.imshow(np.sqrt(diff_2l_alpha))

    rmses.set_title("RMSE Evolution")
    rmses.set_xlim(steps_alpha[0], steps_alpha[-1])
    rmses.set_ylim(bottom=0, top=1.1*max(max(rmse_1l_2l_30), max(rmse_1l_alpha_30), max(rmse_2l_alpha_30)))
    rmses.plot(steps_alpha[:i+1], rmse_1l_2l_30, label = r"1l - 2l")
    rmses.plot(steps_alpha[:i+1], rmse_1l_alpha_30, label = r"1l - $\alpha$")
    rmses.plot(steps_alpha[:i+1], rmse_2l_alpha_30, label = r"2l - $\alpha$")
    rmses.legend()

    plt.pause(0.01)

    if False and (i% freq_save == 0) or  (i == len(files_alpha) - 1):
        fig.savefig(ROOT.joinpath(f"output/local/changing_30/{i}.png"))


plt.ioff()
%matplotlib inline 
plt.figure(figsize=(18,6))
plt.title("RMSE Evolution - Kernel 30")
plt.xlim(steps_alpha[0], steps_alpha[-1])
plt.ylim(bottom=0, top=1.1*max(max(rmse_1l_2l_30), max(rmse_1l_alpha_30), max(rmse_2l_alpha_30)))
plt.plot(steps_alpha[:i+1], rmse_1l_2l_30, label = r"1l - 2l")
plt.plot(steps_alpha[:i+1], rmse_1l_alpha_30, label = r"1l - $\alpha$")
plt.plot(steps_alpha[:i+1], rmse_2l_alpha_30, label = r"2l - $\alpha$")
plt.legend()
plt.show()

# Kernel : 50

In [ ]:
%matplotlib inline

coef = ChangingCoefficient.from_file(ROOT.joinpath("data/coefficients.npz"))
coef._kernel_width = 50
coef._coef_interpolation = coef._interpolate_coefs(coef._times, coef._coefs)
coef.show(False)

In [ ]:
%matplotlib widget

folder_alpha = ROOT.joinpath("output/g5k/changing_kernel_50")
run_alpha = RunSummary.from_file(folder_alpha.joinpath("_summary.toml"))
steps_alpha, files_alpha = sort_files(list(folder_alpha.glob(f"{run_alpha.configuration.model.prefix}*.npz")),run_alpha.configuration.model.prefix,".npz")
folder_1l = ROOT.joinpath("output/g5k/one_layer_baroclinic_100km")
run_1l = RunSummary.from_file(folder_1l.joinpath("_summary.toml"))
steps_1l, files_1l = sort_files(list(folder_1l.glob(f"{run_1l.configuration.model.prefix}*.npz")),run_1l.configuration.model.prefix,".npz")
folder_2l = ROOT.joinpath("output/g5k/two_layers_baroclinic_100km")
run_2l = RunSummary.from_file(folder_2l.joinpath("_summary.toml"))
steps_2l, files_2l = sort_files(list(folder_2l.glob(f"{run_2l.configuration.model.prefix}*.npz")),run_2l.configuration.model.prefix,".npz")

if not (run_1l.dt == run_alpha.dt == run_2l.dt):
    print("Timesteps are not matching.")
if not (run_1l.duration == run_alpha.duration == run_2l.duration):
    print("Duration are not matching.")

plt.ion()
fig = plt.figure(figsize = (18,18))

layer_1 = fig.add_subplot(3,3,1)
layer_2 = fig.add_subplot(3,3,2)
layer_3 = fig.add_subplot(3,3,3)
comparison_1 = fig.add_subplot(3,3,4)
comparison_2 = fig.add_subplot(3,3,5)
comparison_3 = fig.add_subplot(3,3,6)

rmses = fig.add_subplot(3,1,3)

rmse_1l_2l_50 = []
rmse_1l_alpha_50 = []
rmse_2l_alpha_50 = []

freq_save = len(files_alpha) // 10 + 1

for i in range(len(files_alpha)):
    if not (steps_alpha[i] == steps_1l[i] == steps_2l[i]):
        msg = f"Impossible to match steps {steps_alpha[i]},{steps_1l[i]} and {steps_2l[i]}."
        raise ValueError(msg)
    
    layer_alpha = np.load(files_alpha[i])["omega"][0,0]
    layer_1l = np.load(files_1l[i])["omega"][0,0]
    layer_2l = np.load(files_2l[i])["omega"][0,0]

    diff_1l_2l = np.power(layer_1l - layer_2l,2)
    diff_1l_alpha = np.power(layer_1l - layer_alpha,2)
    diff_2l_alpha = np.power(layer_2l - layer_alpha,2)

    rmse_1l_2l_50.append(np.sqrt(np.sum(diff_1l_2l)) )
    rmse_1l_alpha_50.append(np.sqrt(np.sum(diff_1l_alpha)))
    rmse_2l_alpha_50.append(np.sqrt(np.sum(diff_2l_alpha)) )

    
    fig.suptitle(f"t: {steps_alpha[i]*run_alpha.dt}s/{steps_alpha[-1]*run_alpha.dt}s")

    layer_1.cla()
    layer_2.cla()
    layer_3.cla()
    comparison_1.cla()
    comparison_2.cla()
    comparison_3.cla()

    rmses.cla()

    layer_1.set_title("One Layer")
    layer_1.imshow(layer_1l)
    layer_2.set_title("Modified Layer")
    layer_2.imshow(layer_alpha)
    layer_3.set_title("Two Layers")
    layer_3.imshow(layer_2l)
    comparison_1.set_title(fr"1l - 2l: {rmse_1l_2l_50[-1]:.2f}")
    comparison_1.imshow(np.sqrt(diff_1l_2l))
    comparison_2.set_title(rf"1l - $\alpha$: {rmse_1l_alpha_50[-1]:.2f}")
    comparison_2.imshow(np.sqrt(diff_1l_alpha))
    comparison_3.set_title(fr"2l - $\alpha$: {rmse_2l_alpha_50[-1]:.2f}")
    comparison_3.imshow(np.sqrt(diff_2l_alpha))

    rmses.set_title("RMSE Evolution")
    rmses.set_xlim(steps_alpha[0], steps_alpha[-1])
    rmses.set_ylim(bottom=0, top=1.1*max(max(rmse_1l_2l_50), max(rmse_1l_alpha_50), max(rmse_2l_alpha_50)))
    rmses.plot(steps_alpha[:i+1], rmse_1l_2l_50, label = r"1l - 2l")
    rmses.plot(steps_alpha[:i+1], rmse_1l_alpha_50, label = r"1l - $\alpha$")
    rmses.plot(steps_alpha[:i+1], rmse_2l_alpha_50, label = r"2l - $\alpha$")
    rmses.legend()

    plt.pause(0.01)

    if False and (i% freq_save == 0) or  (i == len(files_alpha) - 1):
        fig.savefig(ROOT.joinpath(f"output/local/changing_50/{i}.png"))
        
plt.ioff()
%matplotlib inline 
plt.figure(figsize=(18,6))
plt.title("RMSE Evolution - Kernel 50")
plt.xlim(steps_alpha[0], steps_alpha[-1])
plt.ylim(bottom=0, top=1.1*max(max(rmse_1l_2l_50), max(rmse_1l_alpha_50), max(rmse_2l_alpha_50)))
plt.plot(steps_alpha[:i+1], rmse_1l_2l_50, label = r"1l - 2l")
plt.plot(steps_alpha[:i+1], rmse_1l_alpha_50, label = r"1l - $\alpha$")
plt.plot(steps_alpha[:i+1], rmse_2l_alpha_50, label = r"2l - $\alpha$")
plt.legend()
plt.show()

# Kernel : 100

In [ ]:
%matplotlib inline

coef = ChangingCoefficient.from_file(ROOT.joinpath("data/coefficients.npz"))
coef._kernel_width = 100
coef._coef_interpolation = coef._interpolate_coefs(coef._times, coef._coefs)
coef.show(False)

In [ ]:
%matplotlib widget

folder_alpha = ROOT.joinpath("output/g5k/changing_kernel_100")
run_alpha = RunSummary.from_file(folder_alpha.joinpath("_summary.toml"))
steps_alpha, files_alpha = sort_files(list(folder_alpha.glob(f"{run_alpha.configuration.model.prefix}*.npz")),run_alpha.configuration.model.prefix,".npz")
folder_1l = ROOT.joinpath("output/g5k/one_layer_baroclinic_100km")
run_1l = RunSummary.from_file(folder_1l.joinpath("_summary.toml"))
steps_1l, files_1l = sort_files(list(folder_1l.glob(f"{run_1l.configuration.model.prefix}*.npz")),run_1l.configuration.model.prefix,".npz")
folder_2l = ROOT.joinpath("output/g5k/two_layers_baroclinic_100km")
run_2l = RunSummary.from_file(folder_2l.joinpath("_summary.toml"))
steps_2l, files_2l = sort_files(list(folder_2l.glob(f"{run_2l.configuration.model.prefix}*.npz")),run_2l.configuration.model.prefix,".npz")

if not (run_1l.dt == run_alpha.dt == run_2l.dt):
    print("Timesteps are not matching.")
if not (run_1l.duration == run_alpha.duration == run_2l.duration):
    print("Duration are not matching.")

plt.ion()
fig = plt.figure(figsize = (18,18))

layer_1 = fig.add_subplot(3,3,1)
layer_2 = fig.add_subplot(3,3,2)
layer_3 = fig.add_subplot(3,3,3)
comparison_1 = fig.add_subplot(3,3,4)
comparison_2 = fig.add_subplot(3,3,5)
comparison_3 = fig.add_subplot(3,3,6)

rmses = fig.add_subplot(3,1,3)

rmse_1l_2l_100 = []
rmse_1l_alpha_100 = []
rmse_2l_alpha_100 = []

freq_save = len(files_alpha) // 10 + 1

for i in range(len(files_alpha)):
    if not (steps_alpha[i] == steps_1l[i] == steps_2l[i]):
        msg = f"Impossible to match steps {steps_alpha[i]},{steps_1l[i]} and {steps_2l[i]}."
        raise ValueError(msg)
    
    layer_alpha = np.load(files_alpha[i])["omega"][0,0]
    layer_1l = np.load(files_1l[i])["omega"][0,0]
    layer_2l = np.load(files_2l[i])["omega"][0,0]

    diff_1l_2l = np.power(layer_1l - layer_2l,2)
    diff_1l_alpha = np.power(layer_1l - layer_alpha,2)
    diff_2l_alpha = np.power(layer_2l - layer_alpha,2)

    rmse_1l_2l_100.append(np.sqrt(np.sum(diff_1l_2l)) )
    rmse_1l_alpha_100.append(np.sqrt(np.sum(diff_1l_alpha)))
    rmse_2l_alpha_100.append(np.sqrt(np.sum(diff_2l_alpha)) )

    
    fig.suptitle(f"t: {steps_alpha[i]*run_alpha.dt}s/{steps_alpha[-1]*run_alpha.dt}s")

    layer_1.cla()
    layer_2.cla()
    layer_3.cla()
    comparison_1.cla()
    comparison_2.cla()
    comparison_3.cla()

    rmses.cla()

    layer_1.set_title("One Layer")
    layer_1.imshow(layer_1l)
    layer_2.set_title("Modified Layer")
    layer_2.imshow(layer_alpha)
    layer_3.set_title("Two Layers")
    layer_3.imshow(layer_2l)
    comparison_1.set_title(fr"1l - 2l: {rmse_1l_2l_100[-1]:.2f}")
    comparison_1.imshow(np.sqrt(diff_1l_2l))
    comparison_2.set_title(rf"1l - $\alpha$: {rmse_1l_alpha_100[-1]:.2f}")
    comparison_2.imshow(np.sqrt(diff_1l_alpha))
    comparison_3.set_title(fr"2l - $\alpha$: {rmse_2l_alpha_100[-1]:.2f}")
    comparison_3.imshow(np.sqrt(diff_2l_alpha))

    rmses.set_title("RMSE Evolution")
    rmses.set_xlim(steps_alpha[0], steps_alpha[-1])
    rmses.set_ylim(bottom=0, top=1.1*max(max(rmse_1l_2l_100), max(rmse_1l_alpha_100), max(rmse_2l_alpha_100)))
    rmses.plot(steps_alpha[:i+1], rmse_1l_2l_100, label = r"1l - 2l")
    rmses.plot(steps_alpha[:i+1], rmse_1l_alpha_100, label = r"1l - $\alpha$")
    rmses.plot(steps_alpha[:i+1], rmse_2l_alpha_100, label = r"2l - $\alpha$")
    rmses.legend()

    plt.pause(0.01)

    if False and (i% freq_save == 0) or  (i == len(files_alpha) - 1):
        fig.savefig(ROOT.joinpath(f"output/local/changing_100/{i}.png"))

plt.ioff()
%matplotlib inline 
plt.figure(figsize=(18,6))
plt.title("RMSE Evolution - Kernel 100")
plt.xlim(steps_alpha[0], steps_alpha[-1])
plt.ylim(bottom=0, top=1.1*max(max(rmse_1l_2l_100), max(rmse_1l_alpha_100), max(rmse_2l_alpha_100)))
plt.plot(steps_alpha[:i+1], rmse_1l_2l_100, label = r"1l - 2l")
plt.plot(steps_alpha[:i+1], rmse_1l_alpha_100, label = r"1l - $\alpha$")
plt.plot(steps_alpha[:i+1], rmse_2l_alpha_100, label = r"2l - $\alpha$")
plt.legend()
plt.show()

# Kernel : 300

In [ ]:
%matplotlib inline

coef = ChangingCoefficient.from_file(ROOT.joinpath("data/coefficients.npz"))
coef._kernel_width = 300
coef._coef_interpolation = coef._interpolate_coefs(coef._times, coef._coefs)
coef.show(False)

In [ ]:
%matplotlib widget

folder_alpha = ROOT.joinpath("output/g5k/changing_kernel_300")
run_alpha = RunSummary.from_file(folder_alpha.joinpath("_summary.toml"))
steps_alpha, files_alpha = sort_files(list(folder_alpha.glob(f"{run_alpha.configuration.model.prefix}*.npz")),run_alpha.configuration.model.prefix,".npz")
folder_1l = ROOT.joinpath("output/g5k/one_layer_baroclinic_100km")
run_1l = RunSummary.from_file(folder_1l.joinpath("_summary.toml"))
steps_1l, files_1l = sort_files(list(folder_1l.glob(f"{run_1l.configuration.model.prefix}*.npz")),run_1l.configuration.model.prefix,".npz")
folder_2l = ROOT.joinpath("output/g5k/two_layers_baroclinic_100km")
run_2l = RunSummary.from_file(folder_2l.joinpath("_summary.toml"))
steps_2l, files_2l = sort_files(list(folder_2l.glob(f"{run_2l.configuration.model.prefix}*.npz")),run_2l.configuration.model.prefix,".npz")

if not (run_1l.dt == run_alpha.dt == run_2l.dt):
    print("Timesteps are not matching.")
if not (run_1l.duration == run_alpha.duration == run_2l.duration):
    print("Duration are not matching.")

plt.ion()
fig = plt.figure(figsize = (18,18))

layer_1 = fig.add_subplot(3,3,1)
layer_2 = fig.add_subplot(3,3,2)
layer_3 = fig.add_subplot(3,3,3)
comparison_1 = fig.add_subplot(3,3,4)
comparison_2 = fig.add_subplot(3,3,5)
comparison_3 = fig.add_subplot(3,3,6)

rmses = fig.add_subplot(3,1,3)

rmse_1l_2l_300 = []
rmse_1l_alpha_300 = []
rmse_2l_alpha_300 = []

freq_save = len(files_alpha) // 10 + 1

for i in range(len(files_alpha)):
    if not (steps_alpha[i] == steps_1l[i] == steps_2l[i]):
        msg = f"Impossible to match steps {steps_alpha[i]},{steps_1l[i]} and {steps_2l[i]}."
        raise ValueError(msg)
    
    layer_alpha = np.load(files_alpha[i])["omega"][0,0]
    layer_1l = np.load(files_1l[i])["omega"][0,0]
    layer_2l = np.load(files_2l[i])["omega"][0,0]

    diff_1l_2l = np.power(layer_1l - layer_2l,2)
    diff_1l_alpha = np.power(layer_1l - layer_alpha,2)
    diff_2l_alpha = np.power(layer_2l - layer_alpha,2)

    rmse_1l_2l_300.append(np.sqrt(np.sum(diff_1l_2l)) )
    rmse_1l_alpha_300.append(np.sqrt(np.sum(diff_1l_alpha)))
    rmse_2l_alpha_300.append(np.sqrt(np.sum(diff_2l_alpha)) )

    
    fig.suptitle(f"t: {steps_alpha[i]*run_alpha.dt}s/{steps_alpha[-1]*run_alpha.dt}s")

    layer_1.cla()
    layer_2.cla()
    layer_3.cla()
    comparison_1.cla()
    comparison_2.cla()
    comparison_3.cla()

    rmses.cla()

    layer_1.set_title("One Layer")
    layer_1.imshow(layer_1l)
    layer_2.set_title("Modified Layer")
    layer_2.imshow(layer_alpha)
    layer_3.set_title("Two Layers")
    layer_3.imshow(layer_2l)
    comparison_1.set_title(fr"1l - 2l: {rmse_1l_2l_300[-1]:.2f}")
    comparison_1.imshow(np.sqrt(diff_1l_2l))
    comparison_2.set_title(rf"1l - $\alpha$: {rmse_1l_alpha_300[-1]:.2f}")
    comparison_2.imshow(np.sqrt(diff_1l_alpha))
    comparison_3.set_title(fr"2l - $\alpha$: {rmse_2l_alpha_300[-1]:.2f}")
    comparison_3.imshow(np.sqrt(diff_2l_alpha))

    rmses.set_title("RMSE Evolution")
    rmses.set_xlim(steps_alpha[0], steps_alpha[-1])
    rmses.set_ylim(bottom=0, top=1.1*max(max(rmse_1l_2l_300), max(rmse_1l_alpha_300), max(rmse_2l_alpha_300)))
    rmses.plot(steps_alpha[:i+1], rmse_1l_2l_300, label = r"1l - 2l")
    rmses.plot(steps_alpha[:i+1], rmse_1l_alpha_300, label = r"1l - $\alpha$")
    rmses.plot(steps_alpha[:i+1], rmse_2l_alpha_300, label = r"2l - $\alpha$")
    rmses.legend()

    plt.pause(0.01)

    if False and (i% freq_save == 0) or  (i == len(files_alpha) - 1):
        fig.savefig(ROOT.joinpath(f"output/local/changing_300/{i}.png"))

plt.ioff()
%matplotlib inline 
plt.figure(figsize=(18,6))
plt.title("RMSE Evolution - Kernel 100")
plt.xlim(steps_alpha[0], steps_alpha[-1])
plt.ylim(bottom=0, top=1.1*max(max(rmse_1l_2l_300), max(rmse_1l_alpha_300), max(rmse_2l_alpha_300)))
plt.plot(steps_alpha[:i+1], rmse_1l_2l_300, label = r"1l - 2l")
plt.plot(steps_alpha[:i+1], rmse_1l_alpha_300, label = r"1l - $\alpha$")
plt.plot(steps_alpha[:i+1], rmse_2l_alpha_300, label = r"2l - $\alpha$")
plt.legend()
plt.show()

# Comparisons

In [ ]:
%matplotlib inline

plt.figure(figsize=(18,6))
plt.title("1l - 2l Comparison")
plt.plot(rmse_1l_2l_30, label = "Kernel: 30")
plt.plot(rmse_1l_2l_50, label = "Kernel: 50")
plt.plot(rmse_1l_2l_100, label = "Kernel: 100")
plt.plot(rmse_1l_2l_300, label = "Kernel: 300")
plt.legend()
plt.show()

plt.figure(figsize=(18,6))
plt.title(r"1l - $\alpha$ Comparison")
plt.plot(rmse_1l_alpha_30, label = "Kernel: 30")
plt.plot(rmse_1l_alpha_50, label = "Kernel: 50")
plt.plot(rmse_1l_alpha_100, label = "Kernel: 100")
plt.plot(rmse_1l_alpha_300, label = "Kernel: 300")
plt.legend()
plt.show()

plt.figure(figsize=(18,6))
plt.title(r"2l - $\alpha$ Comparison")
plt.plot(rmse_2l_alpha_30, label = "Kernel: 30")
plt.plot(rmse_2l_alpha_50, label = "Kernel: 50")
plt.plot(rmse_2l_alpha_100, label = "Kernel: 100")
plt.plot(rmse_2l_alpha_300, label = "Kernel: 300")
plt.legend()
plt.show()